In [1]:
import os
from glob import glob

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import mne
mne.set_log_level(verbose=False)

from Datasets import *

In [4]:
BATCH_SIZE = 4
channels = ['F4']

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
%%time
datasets = [EDFData_PTH(path, channels=channels) for path in glob("../Data/*edf")]
dataset_together = torch.utils.data.ConcatDataset(datasets)
dataloaders = [torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, drop_last=True) for dataset in datasets]
dataloader_together = torch.utils.data.DataLoader(dataset_together, batch_size = BATCH_SIZE, drop_last=True)
len(datasets), len(dataloaders), len(dataset_together), len(dataloader_together)

e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 109 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 92 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: RuntimeWarning: Omitted 120 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:18: R

(14, 14, 11157, 2789)

In [7]:
[print(dataset.epochs) for dataset in datasets]
print()

<Epochs |  882 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 25
 'Sleep stage N2': 317
 'Sleep stage N3': 193
 'Sleep stage R': 101
 'Sleep stage W': 246>
<Epochs |  756 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 14
 'Sleep stage N2': 455
 'Sleep stage N3': 114
 'Sleep stage W': 173>
<Epochs |  715 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 4
 'Sleep stage N2': 248
 'Sleep stage N3': 185
 'Sleep stage R': 93
 'Sleep stage W': 185>
<Epochs |  764 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 14
 'Sleep stage N2': 403
 'Sleep stage N3': 218
 'Sleep stage R': 94
 'Sleep stage W': 35>
<Epochs |  805 events (all good), 0 - 29.998 sec, baseline off, ~6 kB, data not loaded,
 'Sleep stage N1': 127
 'Sleep stage N2': 293
 'Sleep stage N3': 169
 'Sleep stage R': 86
 'Sleep stage W': 130>
<Epochs |  780 events

In [8]:
sampling_rate = int(datasets[0].sampling_rate)
sampling_rate

512

In [9]:
class TinySleepNet(nn.Module):
    def __init__(self, sampling_rate, channels):
        super(TinySleepNet, self).__init__()
        self.sampling_rate = sampling_rate
        self.channels = channels

        self.feature_extraction = nn.Sequential(*[
            nn.Conv1d(in_channels=len(channels), out_channels=128, kernel_size=sampling_rate//2, stride=sampling_rate//4),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=8, stride=8),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8, stride=1, padding=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8, stride=1, padding=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        ])

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        
        self.classifier = nn.Linear(2*128, 10)

    def forward(self, X):
        X = self.feature_extraction(X)
        X, _ = self.lstm(X.permute(0,2,1))
        X = self.classifier(X.reshape(X.shape[0],-1))
        return X

In [99]:
model = TinySleepNet(sampling_rate, channels)
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
model

TinySleepNet(
  (feature_extraction): Sequential(
    (0): Conv1d(1, 128, kernel_size=(256,), stride=(128,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(3,))
    (4): ReLU()
    (5): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(3,))
    (6): ReLU()
    (7): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(3,))
    (8): ReLU()
    (9): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(128, 128, batch_first=True)
  (classifier): Linear(in_features=256, out_features=10, bias=True)
)

In [87]:
def train_step(model, optimizer, loss_fn, history, X, Y, metrics=None):
    """
    Trains the model over a batch.

    Parameters
    ----------
    model: nn.Module
        Torch model.
    optimizer:
        Torch optimizer.
    loss_fn:
        Torch loss function.
    X: torch.Tensor
    Y: torch.Tensor
    metrics: dict{str:functions}
        Dict of functions (metrics) we want to calculate and their name.
    
    Returns
    -------
    history: dict{str:float}
        Dictionary of metrics.
    """
    optimizer.zero_grad()
    pred = model(X)
    loss = loss_fn(pred, Y)
    loss.backward()

    history['loss'] = loss.item()
    if metrics:
        with torch.no_grad():
            for name, metric_fn in metrics.items():
                history[name] = metric_fn(pred, Y)

    optimizer.step()
    return history

In [88]:
def val_step(model, optimizer, loss_fn, history, X, Y, metrics=None):
    """
    Calculates the validation metrics over a batch.

    Parameters
    ----------
    model: nn.Module
        Torch model.
    optimizer:
        Torch optimizer.
    loss_fn:
        Torch loss function.
    X: torch.Tensor
    Y: torch.Tensor
    metrics: dict{str:functions}
        Dict of functions (metrics) we want to calculate and their name.
    
    Returns
    -------
    history: dict{str:float}
        Dictionary of metrics.
    """
    with torch.no_grad():
        pred = model(X)
        loss = loss_fn(pred, Y)

        history['val_loss'] = loss.item()
        if metrics:
            for name, metric_fn in metrics.items():
                history['val_'+name] = metric_fn(pred, Y)

    return history

In [51]:
class History():
    """
    Class designed to track the metrics during the training of a NN.
    """
    
    def __init__(self):
        self.history = {}

    def update(self, history):
        """
        Parameters
        ----------
        history: dict{str:float}
        """
        for metric_name, metric_value in history.items():
            if metric_name not in self.history.keys():
                self.history[metric_name] = [metric_value]
            else:
                self.history[metric_name].append(metric_value)

In [83]:
def accuracy_fn(Y_pred, Y_true):
    """
    Calculates the accuracy of our model given its predictions and labels.

    Parameters
    ----------
    Y_pred: torch.Tensor
        Raw output from the nn (logits).
    Y_true: torch.Tensor
        Objective labels.
    
    Returns
    -------
    accuracy: float
    """
    Y_pred = torch.softmax(Y_pred, dim=-1)
    Y_pred = Y_pred.argmax(dim=-1)
    accuracy = torch.where(Y_pred==Y_true, 1, 0).sum() / len(Y_true)

    return accuracy.item()

In [100]:
%%time
EPOCHS = 5
VERBOSE = True

metrics = {
    'accuracy':accuracy_fn
}

history_epoch = History()

for epoch in range(EPOCHS):
    history_batch = History()
    for batch_i, (X, Y) in enumerate(dataloader_together, 1):
        X, Y = X.to(device), torch.squeeze(Y).long().to(device)
        history_train = train_step(model, optimizer, loss_fn, {}, X, Y, metrics)
        history_batch.update(history_train)
        # print(f"Batch {batch_i} -> Loss: {history.history}")
        # if batch_i == 2:
        #     break
    for batch_i, (X, Y) in enumerate(dataloader_together, 1):
        X, Y = X.to(device), torch.squeeze(Y).long().to(device)
        history_val = val_step(model, optimizer, loss_fn, {}, X, Y, metrics)
        history_batch.update(history_val)
        # if batch_i == 2:
        #     break
    history_epoch.update({name:np.mean(values) for name,values in history_batch.history.items()})
    if VERBOSE:
        print(f"Epoch {epoch+1} -> [Train] (Loss) {history_epoch.history['loss'][-1]:.4f} (Acc) {history_epoch.history['accuracy'][-1]:.4f} | [Val] (Loss) {history_epoch.history['val_loss'][-1]:.4f} (Acc) {history_epoch.history['val_accuracy'][-1]:.4f}")
    # break

Epoch 1 -> [Train] (Loss) 1.4944 (Acc) 0.4178 [Val] (Loss) 1.4518 (Acc) 0.2331


KeyboardInterrupt: 

In [101]:
history_batch.history

9669771194458,
  1.4156529903411865,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  1.7293391227722168,
  2.3286783695220

In [102]:
history_epoch.history

{'loss': [1.4943932564491684],
 'accuracy': [0.41780207959842236],
 'val_loss': [1.4518339147957562],
 'val_accuracy': [0.23305844388669775]}